In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Loading data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [ ]:
print(data.isnull().any(axis = 0))

is_sarcastic    False
headline        False
article_link    False
dtype: bool


In [ ]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [ ]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
# getting features and labels
features = data['headline']
labels = data['is_sarcastic']

In [ ]:
print(features)

0        thirtysomething scientists unveil doomsday clo...
1        dem rep  totally nails why congress is falling...
2        eat your veggies    deliciously different recipes
3        inclement weather prevents liar from getting t...
4        mother comes pretty close to using word  strea...
                               ...                        
28614         jews to celebrate rosh hashasha or something
28615    internal affairs investigator disappointed con...
28616    the most beautiful acceptance speech this week...
28617    mars probe destroyed by orbiting spielberg gat...
28618                   dad clarifies this not a food stop
Name: headline, Length: 28619, dtype: object


In [ ]:
# Stemming our data
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
featuresps = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

In [ ]:
lm = WordNetLemmatizer()
featureslm = features.apply(lambda x : ' '.join([lm.lemmatize(word, tag) for word in x]))

In [ ]:
print(featuresps)

0        thirtysometh scientist unveil doomsday clock o...
1        dem rep total nail whi congress is fall short ...
2                       eat your veggi delici differ recip
3          inclement weather prevent liar from get to work
4        mother come pretti close to use word stream co...
                               ...                        
28614                jew to celebr rosh hashasha or someth
28615    intern affair investig disappoint conspiraci d...
28616    the most beauti accept speech thi week came fr...
28617    mar probe destroy by orbit spielberg gate spac...
28618                      dad clarifi thi not a food stop
Name: headline, Length: 28619, dtype: object


In [ ]:
print(featureslm)

0        thirtysomething scientist unveil doomsday cloc...
1        dem rep totally nail why congress is falling s...
2             eat your veggie deliciously different recipe
3        inclement weather prevents liar from getting t...
4        mother come pretty close to using word streami...
                               ...                        
28614          jew to celebrate rosh hashasha or something
28615    internal affair investigator disappointed cons...
28616    the most beautiful acceptance speech this week...
28617    mar probe destroyed by orbiting spielberg gate...
28618                   dad clarifies this not a food stop
Name: headline, Length: 28619, dtype: object


In [ ]:
# vectorizing the data with maximum of 5000 features
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(featureslm)
features = tv.fit_transform(features).toarray()

In [ ]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .20, random_state = 0)

In [ ]:
# model 1:-
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print(lsvc.score(features_train, labels_train)) # 90.93
print(lsvc.score(features_test, labels_test))   # 83.75
# model 2:-
# Using Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print(gnb.score(features_train, labels_train))  # 78.86
print(gnb.score(features_test, labels_test))    # 73.80
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print(lr.score(features_train, labels_train))   # 88.16
print(lr.score(features_test, labels_test))     # 83.08
# model 4:-
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print(rfc.score(features_train, labels_train))  # 98.82
print(rfc.score(features_test, labels_test))    # 79.71


0.9152653417776807
0.8270440251572327
0.8108320594016161
0.7372466806429071
0.8822013540074252
0.8305380852550663
0.9884690980563442
0.7891334730957372
